<a href="https://colab.research.google.com/github/Shen-Xiaoxiao/Shen-Xiaoxiao.github.io/blob/main/Exercises_Chapter_4_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PYTHON NOTEBOOK USED TO ANSWER TO EXERCISES OF CHAPTER 4 OF MATH80624 LECTURE NOTES (Data)

Modified by:
1. Chun Peng (Created for RSOME January 2021)
2. Erick Delage (January 2021)

As discussed in Chapter 4 of the  [lecture notes](http://tintin.hec.ca/pages/erick.delage/MATH80624_LectureNotes.pdf) of MATH80624 at HEC Montréal.

WARNING!!!

The following code exploits a free Mosek licence the course "061652 ROBUST OPTIMIZATION" offered at Politecnico di Milano (expiration April 25th 2024). If you have error messages informing you about licencing issues, you may try uncommenting the installation lines for Gurobi. Otherwise, we recommend that you obtain your own licence of either Mosek ([url](https://www.mosek.com/)) or Gurobi ([url](https://www.gurobi.com/)).

# **Preliminaries**

In [ ]:
!pip install rsome
!pip install mosek
!rm mosek.lic
!git clone https://github.com/erickdelage/80624
!cp ./80624/mosek.lic .
!rm -r ./80624
!mkdir -p /root/mosek
!cp ./mosek.lic /root/mosek
#!pip install -i https://pypi.gurobi.com gurobipy

In [ ]:
import rsome as rso
import numpy as np
from rsome import ro
from rsome import msk_solver as my_solver  #Import Mosek solver interface
#from rsome import grb_solver as my_solver  #Import Gurobi solver interface


### Load the data

In [ ]:
n=4    #The number of facility locations
m=12   #The number of retailer locations
c = np.array([9.1, 8.0, 4.5, 2.1])                                                        #Installation cost for each facility location
r = 2*np.ones((n,m))                                                                      #Retalier price
P = np.array([23, 168, 110, 295])                                                         #Capacity of each facility locations
D_bar = np.array([24, 12, 18, 23, 24, 13, 11, 9, 18, 25, 25, 23])                         #Nominal demand of each retalier
D_hat = np.array([18, 1, 14, 12, 13, 5, 6, 0, 4, 23, 21, 20])                             #Maximum deviation from the nominal demand
d = np.array([[2.31, 2.37, 1.89, 1.92, 1.98, 1.69, 2.37, 2.14, 2.87, 2.16, 2.15, 1.52],   #Transportation cost from facility to retailers
    [1.88, 2.36, 2.02, 2.77, 1.17, 1.45, 2.64, 1.45, 1.83, 1.80, 1.74, 2.42],
    [2.51, 1.73, 3.50, 2.39, 2.51, 2.50, 3.08, 2.36, 2.35, 1.72, 1.47, 2.10],
    [1.71, 2.99, 1.40, 0.96, 1.79, 1.81, 1.89, 2.01, 2.28, 1.71, 2.98, 2.66]])

#Column constraint generation algorithm parameters
big_M = 10000
tolerance = 1e-6

# **Nominal facility location model**

We will study the following model:
\begin{align}
\max_{x,y} & - \sum_{i=1}^n c_i x_i + \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})y_{ij} &&\\
\text{s.t.} & \sum_{i=1}^n y_{ij} \leq D_j &&\forall\,j \in\{1,\cdots,m\}\\
& \sum_{j=1}^m y_{ij} \leq P_i x_i &&\forall\,\;i \in\{1,\cdots,n\}\\
&  x \in \{0,1\}^n,\;y_{ij} \geq 0 &&\forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\},
\end{align}


In [ ]:
#Solve the nominal facility location model

#Create model
model=ro.Model('Nominal facility location problem')
#Define variables
x = model.dvar(n,vtype='B')
y = model.dvar((n,m))          #in million of units

#List the objective and constraints
model.max(-c@x + ((r-d)*y).sum())
model.st(y.sum(axis=0) <= D_bar)       # Maximum demand at each retalier location
model.st(y.sum(axis=1) <= P*x)         # Capacity of each facility location
model.st(y>=0)

#Solve the model
model.solve(my_solver)
opt_obj = model.get()  #in million of dollars
opt_x = x.get()

print('The objective is', opt_obj, 'and the optimal faciliy location is', opt_x)

# **Exercise 4.1) Implementing vertex enumeration**

We are interested in implementing vertex enumeration on the adjustable robust optimization problem:
\begin{align}
\max_{x,y(\cdot)} \;\;& \min_{z\in\mathcal{Z}(\Gamma)}- \sum_{i=1}^n c_i x_i + \sum_{i=1}^n \sum_{j=1}^m (r_{ij}-d_{ij})y_{ij}(z) &&\\
\text{s.t.}\;\; & \sum_{i=1}^n y_{ij}(z) \leq \bar{D}_j+\hat{D}_j z_j &&\forall z\in\mathcal{Z}(\Gamma),\;\forall z\in\mathcal{Z}(\Gamma)\,,\;\forall\,j \in\{1,\cdots,m\}\\
& \sum_{j=1}^m y_{ij}(z) \leq P_i x_i &&\forall z\in\mathcal{Z}(\Gamma),\;\forall\,\;i \in\{1,\cdots,n\}\\
&  x \in \{0,1\}^n,\;y_{ij}(z) \geq 0 &&\forall z\in\mathcal{Z}(\Gamma),\;\forall\,i \in\{1,\cdots,n\}, j \in\{1,\cdots,m\},
\end{align}
where
$$\mathcal{Z}:= \left\{ z\in \mathbb{R}^m\,\middle|\, -1 \leq z \leq 1,\, \sum_i |z_i| \leq \Gamma\right\}$$

In the case of $\Gamma=1$, there are $2m$ vertices to worry about. Specifically, $ \mathcal{Z}(1) = \text{ConvexHull}(\mathcal{Z}_v(1))$ with $\mathcal{Z}_v(1):=\{-e_1,-e_2,\dots,-e_m,e_1,e_2,\dots,e_m\}\;$.

In the case of $\Gamma=m$, the situation is more complicated as there are now $2^m$ vertices to take care of. Specifically, $ \mathcal{Z}(m) = \text{ConvexHull}(\mathcal{Z}_v(m))$ with $\mathcal{Z}_v(m):=\{-1,1\}^m\;$.

### Useful functions

In [ ]:

def dec2base(ns, b):
    #This function convert a list of numbers to a 2-d np.array containing the base b representation of each of them
    def dec2base_int(n, b):    #This funcion is used to convert a digit number n to base b
        results = []
        while True:
          if n < b:
            results.append(n)# = [n] + results
            break
          else:
            results.append(n%b)# = [n%b] + results
            n = n//b
        results.reverse()
        return np.array(results)

    k = len(dec2base_int(max(ns),b))
    result = np.zeros((len(ns),k))
    for i in range(len(ns)):
        x = ns[i]
        tmp = dec2base_int(x, b)
        if len(tmp) < k:
            tmp = np.append(np.zeros(k-len(tmp)),tmp)
        result[i,:] = tmp
    return result

def Vertex_generation(n,Gamma):
    #This function creates an 2-d np.array containing all the vertices of an n dimensional budgeted set
    #Inputs:
    #  n: dimension of set
    #  Gamma : budget
    #Outputs:
    #  z: n \times N np.array containing N vectors representing each vertex

    tmp1 = dec2base(np.arange(3**n), 3)-1   #Create list of all combinations of {-1, 0, 1}^N                #convert digit to base 3 and convert a set of numbers' digit to array
    tmp2=np.sum(np.abs(tmp1),axis=1)        #Calculate amount of perturbation in each exteme point
    tsce=np.where(tmp2-Gamma==0)[0]      #find where is Gamma constraint active
    Zs=tmp1[tsce,:]
    Zs=np.transpose(Zs)
    N =np.size(Zs,axis=1)
    return (Zs, N)


### Solution:



In [ ]:
#Budget of uncertainty
Gamma = 1
(zbar,size_z) = Vertex_generation(m,Gamma)

# CODE MISSING HERE


opt_obj_41_G1 = ??? # include optimal value here
opt_x_41_G1 = ??? # include solution here
print('when Gamma=', Gamma, 'the objective is', np.round(opt_obj_41_G1,decimals=4), 'and the optimal facility locations are',opt_x_41_G1)

In [ ]:
#Budget of uncertainty
Gamma = m
(zbar,size_z) = Vertex_generation(m,Gamma)

# CODE MISSING HERE


opt_obj_41_Gm = ??? # include optimal value here
opt_x_41_Gm = ??? # include solution here
print('when Gamma=', Gamma, 'the objective is', np.round(opt_obj_41_Gm,decimals=4), 'and the optimal facility locations are',opt_x_41_Gm)

# **Exercise 4.3) Implementing column-and-constraint generation**

Solve the robust two-stage optimization problem presented in problem (4.18) using column-and-constraint generation for the budgeted uncertainty set when $\Gamma=4$.

### Solution:


In [ ]:
#Budget of uncertainty
Gamma = 4

# CODE MISSING HERE


opt_obj_43 = ??? # include optimal value here
opt_x_43 = ??? # include solution here
print('when Gamma=', Gamma, 'the objective is', np.round(opt_obj_43,decimals=4), 'and the optimal facility locations are',opt_x_43)